In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, TFBertModel
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
cwd = os.getcwd()

In [3]:
test_model = tf.saved_model.load('model_2_2023-11-08')

In [5]:
test_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x23e3e4c2f40>

In [9]:
cwd = os.getcwd()
datapath = os.path.join(cwd, 'dataset')

df = pd.read_excel(os.path.join(datapath, 'all_cleaned.xlsx'))
df = df[['berita', 'label']]

df.drop_duplicates(inplace=True)
df.dropna(inplace = True)

df_test = df.sample(10)

max_len = 70
model_name = 'indobenchmark/indobert-base-p1'
tokenizer = AutoTokenizer.from_pretrained(model_name)

X_train = tokenizer(
    text=df_test['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_train

{'input_ids': <tf.Tensor: shape=(10, 70), dtype=int32, numpy=
array([[    2, 13969,  9081,  5490,  4179,  1824,   107, 10334,  3022,
         1179, 18848,   152,   880, 30355,   959,  5237,  9363,   880,
        30355,  7121, 25690,  1867,  5490,  4179,  1824,   107, 10334,
          150,  2318,  2499,  1181,  2690,   150,  3987,  5552, 18303,
        15572,  1480,  3692,  5035,   880, 30355,   678,  4825,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0],
       [    2, 28211, 28819,  5115,  3870, 29912, 18275, 11736,   839,
         1908,   405,  3518,  4891,  3650, 12893,   494,   500,   342,
        15198,  8707,  4431,    55, 14752, 29691, 15809, 30371,   136,
        11864, 27303,  5765, 30362, 25815,  5103,   639,  5115,  1845,
         2106,  8485,   300, 14510,  2276, 25148,    62,   406,  1181,
          405,  2106,    92,

In [14]:
input_ids = tf.TensorSpec.from_tensor(X_train['input_ids'], name = 'input_ids')
attention_mask = tf.TensorSpec.from_tensor(X_train['attention_mask'], name = 'attention_mask')

In [17]:
predicted = test_model([X_train['input_ids'], X_train['attention_mask']])
y_predicted = [1 if pred >= 0.5 else 0 for pred in predicted]
y_predicted

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [19]:
df_test['label']

10494    0
13442    0
23845    1
20227    0
2490     0
2876     0
13431    0
17380    0
20566    0
2228     0
Name: label, dtype: int64